In [46]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests


In [47]:
#download data
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [48]:
#create dataframe
column_names = ['Postcode','Borough','Neighbourhood']
data = pd.DataFrame(columns = column_names)


In [49]:
# loop soup to find postcode, borough, neighborhood 
content = soup.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    data = data.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [50]:
# clean dataframe 
data = data[data.Borough!='Not assigned']
data = data[data.Borough!= 0]
data.reset_index(drop = True, inplace = True)

#fix the not assigned neighborhood
data.Neighborhood.replace('Not assigned',"Queen's Park",inplace=True)



In [51]:
df = data.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [52]:
df.shape

(103, 3)